In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import ElasticNet
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor

import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# **Selección de Características**

Para optimizar el rendimiento del modelo y reducir la dimensionalidad del conjunto de datos, se llevará a cabo un *proceso iterativo de selección de características* basado en la evaluación de un modelo `Random Forest` con su configuración por defecto. Partiendo de un conjunto inicial de casi 400 variables, se implementará una estrategia de adición secuencial de grupos de características.

En cada iteración, se agregará un nuevo grupo de variables al conjunto de características seleccionadas hasta ese momento. Se entrena un modelo Random Forest con la configuración por defecto utilizando este conjunto ampliado de variables y se evalará su rendimiento, comparando las distintas métricas.

Si la inclusión del nuevo grupo de variables resulta en una disminución de las métricas (es decir, una mejora en el rendimiento), estas variables se incorporarán permanentemente al conjunto de características seleccionadas. En caso contrario, el grupo de variables se descartará.

Este proceso iterativo se continuará hasta que no se observen mejoras significativas en las métricas tras la adición de nuevos grupos de variables. 

**Conclusión**: Como resultado de este procedimiento, se obtendrá un conjunto final de 200 características, que constituyen el subconjunto de variables más relevantes para la predicción de la variable objetivo según el criterio del modelo Random Forest.

# 1

In [237]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 60000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1311, 397)


In [238]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto',	'Age_Grouped_Joven','Age_Grouped_Senior']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [239]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.1393
MSE (promedio CV): 117021292.2306
RMSE (promedio CV): 10817.6380
MAE (promedio CV): 8768.3465


# 2

In [240]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [242]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [243]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.1491
MSE (promedio CV): 443447320.7571
RMSE (promedio CV): 21058.1889
MAE (promedio CV): 15482.9814


# 3

In [244]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [245]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [246]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [247]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.1761
MSE (promedio CV): 429467697.4028
RMSE (promedio CV): 20723.6024
MAE (promedio CV): 15099.4454


# 4

In [47]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [48]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [49]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment',
        'Remote_grouped_In-person', 'Remote_grouped_Remote/Hybrid']]

# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.1918
MSE (promedio CV): 421458802.9418
RMSE (promedio CV): 20529.4618
MAE (promedio CV): 14972.0026


**'Remote_grouped_In-person', 'Remote_grouped_Remote/Hybrid':**

Empeora el modelo con esas dos features, las quito y sigo seleccionando otras.

# 5

In [51]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [52]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [53]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.2609
MSE (promedio CV): 385382069.6621
RMSE (promedio CV): 19631.1505
MAE (promedio CV): 14506.1395


El MAE empeora ligeramente, pero el R2 mejora considerablemente, y el RMSE mejora ligeramente. De momento, me dejo el cambio y sigo probando con mas columnas. 

# 6

In [55]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [56]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [57]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.2764
MSE (promedio CV): 377391934.9158
RMSE (promedio CV): 19426.5781
MAE (promedio CV): 14403.7841


La unica metrica que mejora es el R2, me quedo con el cambio

# 7

In [59]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [60]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [61]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.3053
MSE (promedio CV): 362209942.2490
RMSE (promedio CV): 19031.8140
MAE (promedio CV): 14181.3003


El R2 sigue subiendo de a poco, el resto de las metricas siguen practicamente igual. Me quedo con el cambio

# 8

In [63]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [65]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [66]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5394
MSE (promedio CV): 240129596.5417
RMSE (promedio CV): 15496.1155
MAE (promedio CV): 10655.8697


El R2 ha mejorado notablemente al igual que el resto de las metricas. Sigo probando.

# 9

In [67]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [69]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 'DevType_encoded']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [70]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5536
MSE (promedio CV): 232662737.3433
RMSE (promedio CV): 15253.2861
MAE (promedio CV): 10502.0323


Sigue mejorando el R2, aunque las otras empeoran muy ligeramente. Me quedo con los cambios. 

# 10

In [71]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [73]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 'DevType_encoded', 'TimeSearching']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [74]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5525
MSE (promedio CV): 233231912.3592
RMSE (promedio CV): 15271.9322
MAE (promedio CV): 10493.4981


El modelo no mejora ni empeora. Probare agregando una variable muy parecida a esta, si no funcionan las quito para no acumular columnas que no sumen. 

# 11

In [75]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [77]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 'DevType_encoded', 'TimeSearching', 'TimeAnswering']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [78]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5548
MSE (promedio CV): 232027533.7557
RMSE (promedio CV): 15232.4500
MAE (promedio CV): 10500.2406


El modelo no mejora ni empeora con estas variables. Las quito para no acumular variables que no suman. 

# 12

In [79]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [81]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 'DevType_encoded', 'PurchaseInfluence']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [82]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5537
MSE (promedio CV): 232565097.5225
RMSE (promedio CV): 15250.0852
MAE (promedio CV): 10532.2729


Mismo: no suma ni empeora. De momento la quito. 

# 13

In [83]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [85]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 'Frequency_1','Frequency_2','Frequency_3', 'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 'CodingActivities_encoded',
        'AISelect', 'AISent', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 'DevType_encoded', 'OrgSize_Grouped_Desconocido', 'OrgSize_Grouped_Grande','OrgSize_Grouped_Mediana','OrgSize_Grouped_Muy Pequeña','OrgSize_Grouped_Pequeña']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [86]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5519
MSE (promedio CV): 233480462.9407
RMSE (promedio CV): 15280.0675
MAE (promedio CV): 10515.4868


No mejora ni empeora. De momento las quito. 

# 14

In [91]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [92]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [93]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [94]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5535
MSE (promedio CV): 232685799.4858
RMSE (promedio CV): 15254.0421
MAE (promedio CV): 10504.9802


No mejora ni empeora. Pruebo sumando las otras dos columnas de esta pregunta. 

# 15

In [95]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [96]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [97]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [98]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5528
MSE (promedio CV): 232966355.4156
RMSE (promedio CV): 15263.2354
MAE (promedio CV): 10520.4872


Igual. Sumo columna de Mac

# 16

In [99]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [101]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'MacOS']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [102]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5641
MSE (promedio CV): 227239776.9965
RMSE (promedio CV): 15074.4744
MAE (promedio CV): 10371.4234


Mejora. Sumo Windows

# 17

In [103]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [105]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [106]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5777
MSE (promedio CV): 220131084.8608
RMSE (promedio CV): 14836.8152
MAE (promedio CV): 10287.6613


Mejora. Sumo "Otros"

# 18

In [107]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [108]:
df

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net),LanguageW

In [109]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'MacOS', 'Otros']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [110]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5628
MSE (promedio CV): 227939590.4424
RMSE (promedio CV): 15097.6684
MAE (promedio CV): 10354.8958


Empeora. La quito. 

# 19

In [111]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [113]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [114]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5784
MSE (promedio CV): 219841806.2913
RMSE (promedio CV): 14827.0633
MAE (promedio CV): 10273.4138


Mejora el R2 ligeramente. Las dejo. 

# 20

In [115]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [118]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [119]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5815
MSE (promedio CV): 218283886.0221
RMSE (promedio CV): 14774.4335
MAE (promedio CV): 10213.0881


Mejora. Me quedo con los cambios

# 21

In [122]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [125]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [126]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5867
MSE (promedio CV): 215559352.9880
RMSE (promedio CV): 14681.9397
MAE (promedio CV): 10081.0219


Mejora. Los dejo, pero primero pruebo con los lenguajes mas importantes.

# 22

In [127]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [131]:
# Lista de columnas a analizar
columns_to_check = [
    'LanguageHaveWorkedWith_', 'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
    'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 'LanguageHaveWorkedWith_C#', 'LanguageHaveWorkedWith_C++', 
    'LanguageHaveWorkedWith_Clojure', 'LanguageHaveWorkedWith_Cobol', 'LanguageHaveWorkedWith_Crystal', 'LanguageHaveWorkedWith_Dart',
    'LanguageHaveWorkedWith_Delphi', 'LanguageHaveWorkedWith_Elixir', 'LanguageHaveWorkedWith_Erlang', 'LanguageHaveWorkedWith_F#',
    'LanguageHaveWorkedWith_Fortran', 'LanguageHaveWorkedWith_GDScript', 'LanguageHaveWorkedWith_Go', 'LanguageHaveWorkedWith_Groovy',
    'LanguageHaveWorkedWith_HTML/CSS', 'LanguageHaveWorkedWith_Haskell', 'LanguageHaveWorkedWith_Java', 'LanguageHaveWorkedWith_JavaScript',
    'LanguageHaveWorkedWith_Julia', 'LanguageHaveWorkedWith_Kotlin', 'LanguageHaveWorkedWith_Lisp', 'LanguageHaveWorkedWith_Lua', 
    'LanguageHaveWorkedWith_MATLAB', 'LanguageHaveWorkedWith_Nim', 'LanguageHaveWorkedWith_OCaml', 'LanguageHaveWorkedWith_Objective-C',
    'LanguageHaveWorkedWith_PHP', 'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
    'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 'LanguageHaveWorkedWith_Rust',
    'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift',
    'LanguageHaveWorkedWith_TypeScript', 'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 'LanguageHaveWorkedWith_Zig'
]

# Sumar los valores 1 en cada columna seleccionada
language_counts = df[columns_to_check].sum()

# Ordenar de mayor a menor
language_counts_sorted = language_counts.sort_values(ascending=False)

# Mostrar el resultado
print(language_counts_sorted)

LanguageHaveWorkedWith_JavaScript                 1081
LanguageHaveWorkedWith_SQL                         908
LanguageHaveWorkedWith_HTML/CSS                    826
LanguageHaveWorkedWith_Python                      789
LanguageHaveWorkedWith_TypeScript                  691
LanguageHaveWorkedWith_Java                        604
LanguageHaveWorkedWith_Bash/Shell (all shells)     567
LanguageHaveWorkedWith_C#                          376
LanguageHaveWorkedWith_PHP                         368
LanguageHaveWorkedWith_C++                         243
LanguageHaveWorkedWith_C                           222
LanguageHaveWorkedWith_Go                          205
LanguageHaveWorkedWith_PowerShell                  180
LanguageHaveWorkedWith_Kotlin                      173
LanguageHaveWorkedWith_Rust                        127
LanguageHaveWorkedWith_Ruby                        113
LanguageHaveWorkedWith_R                            80
LanguageHaveWorkedWith_Groovy                       80
LanguageHa

In [133]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_SQL','LanguageHaveWorkedWith_HTML/CSS','LanguageHaveWorkedWith_Python',
        'LanguageHaveWorkedWith_TypeScript','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_Bash/Shell (all shells)',
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_PHP','LanguageHaveWorkedWith_C++',
        'LanguageHaveWorkedWith_C','LanguageHaveWorkedWith_Go', 'LanguageHaveWorkedWith_PowerShell',
        'LanguageHaveWorkedWith_Kotlin', 'LanguageHaveWorkedWith_Rust','LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_R','LanguageHaveWorkedWith_Groovy']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [134]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5852
MSE (promedio CV): 216316965.5747
RMSE (promedio CV): 14707.7179
MAE (promedio CV): 10123.4929


Empeora ligeramente, mejor dejo todos los lenguajes y sigo agregando. 

# 23

In [135]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [139]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [140]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5934
MSE (promedio CV): 212073513.7270
RMSE (promedio CV): 14562.7440
MAE (promedio CV): 10002.0028


Mejora. Los dejo

# 24

In [248]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 60000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1311, 397)


In [250]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [252]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores=[]
# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100  
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)  # Guardar MAPE
# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores)  # Promedio de MAPE
# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")  # Imprimir MAPE promedio

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.4804
MSE (promedio CV): 70638595.3480
RMSE (promedio CV): 8404.6770
MAE (promedio CV): 6402.9041
MAPE (promedio CV): 17.2193%


Mejora ligeramente

# 25

In [145]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [147]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'PlatformHaveWorkedWith_', 'PlatformHaveWorkedWith_Amazon Web Services (AWS)', 
        'PlatformHaveWorkedWith_Cloudflare', 'PlatformHaveWorkedWith_Colocation', 
        'PlatformHaveWorkedWith_Digital Ocean', 'PlatformHaveWorkedWith_Firebase', 
        'PlatformHaveWorkedWith_Fly.io', 'PlatformHaveWorkedWith_Google Cloud', 
        'PlatformHaveWorkedWith_Heroku', 'PlatformHaveWorkedWith_Hetzner', 
        'PlatformHaveWorkedWith_IBM Cloud Or Watson', 'PlatformHaveWorkedWith_Linode, now Akamai', 
        'PlatformHaveWorkedWith_Managed Hosting', 'PlatformHaveWorkedWith_Microsoft Azure', 
        'PlatformHaveWorkedWith_Netlify', 'PlatformHaveWorkedWith_OVH', 'PlatformHaveWorkedWith_OpenShift',
        'PlatformHaveWorkedWith_OpenStack', 'PlatformHaveWorkedWith_Oracle Cloud Infrastructure (OCI)', 
        'PlatformHaveWorkedWith_Render', 'PlatformHaveWorkedWith_Scaleway', 'PlatformHaveWorkedWith_VMware',
        'PlatformHaveWorkedWith_Vercel', 'PlatformHaveWorkedWith_Vultr']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [148]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5864
MSE (promedio CV): 215802931.7399
RMSE (promedio CV): 14690.2325
MAE (promedio CV): 10065.5204


Empeora ligeramente. Pruebo solo con las plataformas que mas aparecen en la encuesta. Si no mejora, elimino estas columnas. 

# 26

In [155]:
# Lista de columnas a analizar
columns_to_check = ['PlatformHaveWorkedWith_', 'PlatformHaveWorkedWith_Amazon Web Services (AWS)', 
        'PlatformHaveWorkedWith_Cloudflare', 'PlatformHaveWorkedWith_Colocation', 
        'PlatformHaveWorkedWith_Digital Ocean', 'PlatformHaveWorkedWith_Firebase', 
        'PlatformHaveWorkedWith_Fly.io', 'PlatformHaveWorkedWith_Google Cloud', 
        'PlatformHaveWorkedWith_Heroku', 'PlatformHaveWorkedWith_Hetzner', 
        'PlatformHaveWorkedWith_IBM Cloud Or Watson', 'PlatformHaveWorkedWith_Linode, now Akamai', 
        'PlatformHaveWorkedWith_Managed Hosting', 'PlatformHaveWorkedWith_Microsoft Azure', 
        'PlatformHaveWorkedWith_Netlify', 'PlatformHaveWorkedWith_OVH', 'PlatformHaveWorkedWith_OpenShift',
        'PlatformHaveWorkedWith_OpenStack', 'PlatformHaveWorkedWith_Oracle Cloud Infrastructure (OCI)', 
        'PlatformHaveWorkedWith_Render', 'PlatformHaveWorkedWith_Scaleway', 'PlatformHaveWorkedWith_VMware',
        'PlatformHaveWorkedWith_Vercel', 'PlatformHaveWorkedWith_Vultr']

# Sumar los valores 1 en cada columna seleccionada
language_counts = df[columns_to_check].sum()

# Ordenar de mayor a menor
language_counts_sorted = language_counts.sort_values(ascending=False)

# Mostrar el resultado
check = language_counts_sorted.head(12)
check

PlatformHaveWorkedWith_Amazon Web Services (AWS)    817
PlatformHaveWorkedWith_                             451
PlatformHaveWorkedWith_Microsoft Azure              380
PlatformHaveWorkedWith_Google Cloud                 368
PlatformHaveWorkedWith_Firebase                     216
PlatformHaveWorkedWith_Cloudflare                   196
PlatformHaveWorkedWith_Digital Ocean                157
PlatformHaveWorkedWith_Vercel                       149
PlatformHaveWorkedWith_Heroku                       137
PlatformHaveWorkedWith_VMware                       104
PlatformHaveWorkedWith_Netlify                      104
PlatformHaveWorkedWith_OVH                          102
dtype: int64

In [156]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [158]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'PlatformHaveWorkedWith_Amazon Web Services (AWS)', 'PlatformHaveWorkedWith_', 
        'PlatformHaveWorkedWith_Microsoft Azure', 'PlatformHaveWorkedWith_Google Cloud', 
        'PlatformHaveWorkedWith_Firebase', 'PlatformHaveWorkedWith_Cloudflare', 
        'PlatformHaveWorkedWith_Digital Ocean', 'PlatformHaveWorkedWith_Vercel',
        'PlatformHaveWorkedWith_Heroku', 'PlatformHaveWorkedWith_VMware','PlatformHaveWorkedWith_Netlify',
        'PlatformHaveWorkedWith_OVH']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [159]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5875
MSE (promedio CV): 215233339.0381
RMSE (promedio CV): 14670.8329
MAE (promedio CV): 10051.3025


No mejora. Quito estas columnas. 

# 27

In [160]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [164]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'WebframeHaveWorkedWith_', 'WebframeHaveWorkedWith_ASP.NET', 'WebframeHaveWorkedWith_ASP.NET CORE', 
        'WebframeHaveWorkedWith_Angular', 'WebframeHaveWorkedWith_AngularJS', 'WebframeHaveWorkedWith_Blazor', 
        'WebframeHaveWorkedWith_CodeIgniter', 'WebframeHaveWorkedWith_Deno', 'WebframeHaveWorkedWith_Django', 
        'WebframeHaveWorkedWith_Drupal', 'WebframeHaveWorkedWith_Elm', 'WebframeHaveWorkedWith_Express', 
        'WebframeHaveWorkedWith_FastAPI', 'WebframeHaveWorkedWith_Fastify', 'WebframeHaveWorkedWith_Flask', 
        'WebframeHaveWorkedWith_Gatsby', 'WebframeHaveWorkedWith_Laravel', 'WebframeHaveWorkedWith_NestJS', 
        'WebframeHaveWorkedWith_Next.js', 'WebframeHaveWorkedWith_Node.js', 'WebframeHaveWorkedWith_Nuxt.js', 
        'WebframeHaveWorkedWith_Phoenix', 'WebframeHaveWorkedWith_Play Framework', 'WebframeHaveWorkedWith_React', 
        'WebframeHaveWorkedWith_Remix', 'WebframeHaveWorkedWith_Ruby on Rails', 'WebframeHaveWorkedWith_Solid.js', 
        'WebframeHaveWorkedWith_Spring Boot', 'WebframeHaveWorkedWith_Svelte', 'WebframeHaveWorkedWith_Symfony', 
        'WebframeHaveWorkedWith_Vue.js', 'WebframeHaveWorkedWith_WordPress', 'WebframeHaveWorkedWith_jQuery']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [165]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5895
MSE (promedio CV): 214131707.1111
RMSE (promedio CV): 14633.2398
MAE (promedio CV): 10063.1965


No mejora. Las quito. 

# 28

In [171]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [175]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'MiscTechHaveWorkedWith_', 'MiscTechHaveWorkedWith_.NET (5+) ', 'MiscTechHaveWorkedWith_.NET Framework (1.0 - 4.8)', 
        'MiscTechHaveWorkedWith_.NET MAUI', 'MiscTechHaveWorkedWith_Apache Kafka', 'MiscTechHaveWorkedWith_Apache Spark','MiscTechHaveWorkedWith_CUDA',
        'MiscTechHaveWorkedWith_Capacitor', 'MiscTechHaveWorkedWith_Cordova', 'MiscTechHaveWorkedWith_Electron',
        'MiscTechHaveWorkedWith_Flutter', 'MiscTechHaveWorkedWith_GTK', 'MiscTechHaveWorkedWith_Hadoop', 
        'MiscTechHaveWorkedWith_Hugging Face Transformers', 'MiscTechHaveWorkedWith_Ionic', 'MiscTechHaveWorkedWith_JAX', 
        'MiscTechHaveWorkedWith_Keras', 'MiscTechHaveWorkedWith_Ktor', 'MiscTechHaveWorkedWith_MFC', 
        'MiscTechHaveWorkedWith_NumPy', 'MiscTechHaveWorkedWith_OpenGL', 
        'MiscTechHaveWorkedWith_Opencv', 'MiscTechHaveWorkedWith_Pandas', 'MiscTechHaveWorkedWith_Qt', 
        'MiscTechHaveWorkedWith_Quarkus', 'MiscTechHaveWorkedWith_RabbitMQ', 'MiscTechHaveWorkedWith_React Native', 
        'MiscTechHaveWorkedWith_Scikit-Learn', 'MiscTechHaveWorkedWith_Spring Framework', 'MiscTechHaveWorkedWith_SwiftUI', 
        'MiscTechHaveWorkedWith_Tauri', 'MiscTechHaveWorkedWith_TensorFlow', 'MiscTechHaveWorkedWith_Tidyverse',
        'MiscTechHaveWorkedWith_Torch/PyTorch', 'MiscTechHaveWorkedWith_Xamarin']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [176]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5918
MSE (promedio CV): 212883885.9142
RMSE (promedio CV): 14590.5410
MAE (promedio CV): 10010.3806


No mejora. Las quito. 

# 29

In [177]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [179]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
          'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [180]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5941
MSE (promedio CV): 211685540.0328
RMSE (promedio CV): 14549.4172
MAE (promedio CV): 9992.2734


No mejora pero tampoco empeora el modelo 24 que era el mejor hasta ahora. Lo dejo de momento.  

# 30

In [181]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [183]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
        'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm',
        'NEWCollabToolsHaveWorkedWith_', 'NEWCollabToolsHaveWorkedWith_Android Studio', 
        'NEWCollabToolsHaveWorkedWith_BBEdit', 'NEWCollabToolsHaveWorkedWith_CLion', 
        'NEWCollabToolsHaveWorkedWith_Code::Blocks', 'NEWCollabToolsHaveWorkedWith_DataGrip', 
        'NEWCollabToolsHaveWorkedWith_Eclipse', 'NEWCollabToolsHaveWorkedWith_Emacs', 'NEWCollabToolsHaveWorkedWith_Fleet', 
        'NEWCollabToolsHaveWorkedWith_Geany', 'NEWCollabToolsHaveWorkedWith_Goland', 'NEWCollabToolsHaveWorkedWith_Helix', 
        'NEWCollabToolsHaveWorkedWith_IPython', 'NEWCollabToolsHaveWorkedWith_IntelliJ IDEA', 'NEWCollabToolsHaveWorkedWith_Jupyter Notebook/JupyterLab',
        'NEWCollabToolsHaveWorkedWith_Kate', 'NEWCollabToolsHaveWorkedWith_Nano', 'NEWCollabToolsHaveWorkedWith_Neovim', 
        'NEWCollabToolsHaveWorkedWith_Netbeans', 'NEWCollabToolsHaveWorkedWith_Notepad++', 'NEWCollabToolsHaveWorkedWith_PhpStorm', 
        'NEWCollabToolsHaveWorkedWith_PyCharm', 'NEWCollabToolsHaveWorkedWith_Qt Creator', 'NEWCollabToolsHaveWorkedWith_RStudio', 
        'NEWCollabToolsHaveWorkedWith_Rad Studio (Delphi, C++ Builder)', 'NEWCollabToolsHaveWorkedWith_Rider', 
        'NEWCollabToolsHaveWorkedWith_RubyMine', 'NEWCollabToolsHaveWorkedWith_Spyder', 'NEWCollabToolsHaveWorkedWith_Sublime Text', 
        'NEWCollabToolsHaveWorkedWith_VSCodium', 'NEWCollabToolsHaveWorkedWith_Vim', 'NEWCollabToolsHaveWorkedWith_Visual Studio', 
        'NEWCollabToolsHaveWorkedWith_Visual Studio Code', 'NEWCollabToolsHaveWorkedWith_WebStorm', 'NEWCollabToolsHaveWorkedWith_Xcode']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [184]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5900
MSE (promedio CV): 213805327.3664
RMSE (promedio CV): 14622.0836
MAE (promedio CV): 10006.5360


No mejora. Las quito.

# 31

In [185]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [189]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
        'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm',
        'OfficeStackAsyncHaveWorkedWith_', 'OfficeStackAsyncHaveWorkedWith_Airtable', 
        'OfficeStackAsyncHaveWorkedWith_Asana', 'OfficeStackAsyncHaveWorkedWith_Azure Devops', 
        'OfficeStackAsyncHaveWorkedWith_Basecamp', 'OfficeStackAsyncHaveWorkedWith_Clickup', 
        'OfficeStackAsyncHaveWorkedWith_Confluence', 'OfficeStackAsyncHaveWorkedWith_Doxygen', 
        'OfficeStackAsyncHaveWorkedWith_GitHub Discussions', 'OfficeStackAsyncHaveWorkedWith_Jira', 
        'OfficeStackAsyncHaveWorkedWith_Linear', 'OfficeStackAsyncHaveWorkedWith_Markdown File', 
        'OfficeStackAsyncHaveWorkedWith_Microsoft Lists', 'OfficeStackAsyncHaveWorkedWith_Microsoft Planner',
        'OfficeStackAsyncHaveWorkedWith_Miro', 'OfficeStackAsyncHaveWorkedWith_Monday.com', 
        'OfficeStackAsyncHaveWorkedWith_Notion', 'OfficeStackAsyncHaveWorkedWith_Redmine', 'OfficeStackAsyncHaveWorkedWith_Shortcut', 
        'OfficeStackAsyncHaveWorkedWith_Smartsheet', 'OfficeStackAsyncHaveWorkedWith_Stack Overflow for Teams', 
        'OfficeStackAsyncHaveWorkedWith_Trello', 'OfficeStackAsyncHaveWorkedWith_Wikis', 'OfficeStackAsyncHaveWorkedWith_YouTrack']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [190]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5887
MSE (promedio CV): 214456121.2127
RMSE (promedio CV): 14644.3204
MAE (promedio CV): 10066.1597


Emperora. Los quito. 

# 32

In [191]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [193]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
        'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm', 
        'OfficeStackSyncHaveWorkedWith_', 'OfficeStackSyncHaveWorkedWith_Cisco Webex Teams', 
        'OfficeStackSyncHaveWorkedWith_Discord', 'OfficeStackSyncHaveWorkedWith_Google Chat', 'OfficeStackSyncHaveWorkedWith_Google Meet', 
        'OfficeStackSyncHaveWorkedWith_IRC', 'OfficeStackSyncHaveWorkedWith_Jitsi', 'OfficeStackSyncHaveWorkedWith_Matrix', 
        'OfficeStackSyncHaveWorkedWith_Mattermost', 'OfficeStackSyncHaveWorkedWith_Microsoft Teams', 'OfficeStackSyncHaveWorkedWith_Ringcentral', 
        'OfficeStackSyncHaveWorkedWith_Rocketchat', 'OfficeStackSyncHaveWorkedWith_Signal', 
        'OfficeStackSyncHaveWorkedWith_Skype', 'OfficeStackSyncHaveWorkedWith_Slack', 
        'OfficeStackSyncHaveWorkedWith_Symphony', 'OfficeStackSyncHaveWorkedWith_Telegram', 'OfficeStackSyncHaveWorkedWith_Unify Circuit', 'OfficeStackSyncHaveWorkedWith_Whatsapp', 'OfficeStackSyncHaveWorkedWith_Wickr', 'OfficeStackSyncHaveWorkedWith_Wire', 'OfficeStackSyncHaveWorkedWith_Zoom', 'OfficeStackSyncHaveWorkedWith_Zulip'
        ]]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [194]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5869
MSE (promedio CV): 215420343.3818
RMSE (promedio CV): 14677.2049
MAE (promedio CV): 10064.1789


Empeora. Las quito. 

# 33

In [195]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


In [197]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
        'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm', 
        'AIBen_', 'AIToolCurrently Using_', 'AIToolCurrently Using_Committing and reviewing code', 'AIToolCurrently Using_Debugging and getting help', 
        'AIToolCurrently Using_Deployment and monitoring', 'AIToolCurrently Using_Documenting code', 'AIToolCurrently Using_Learning about a codebase', 
        'AIToolCurrently Using_Project planning', 'AIToolCurrently Using_Testing code', 'AIToolCurrently Using_Writing code']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [198]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5932
MSE (promedio CV): 212154422.2641
RMSE (promedio CV): 14565.5217
MAE (promedio CV): 10002.2484


No mejoran, las quito. 

In [199]:
# DF definitivo:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
        'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm']]

Pruebo nuevamente quitando algunas columnas mas

In [205]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 120000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1801, 397)


Pruebo quitando 'CodingActivities_encoded'

In [207]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
          'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [208]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5727
MSE (promedio CV): 222791503.3862
RMSE (promedio CV): 14926.2019
MAE (promedio CV): 10192.6645


No mejora. Lo dejo

In [253]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 60000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1311, 397)
